In [1]:
import pandas as pd
import numpy as np
import urllib3
import matplotlib.pyplot as plt
import requests
from urllib.parse import urlencode
import time
%matplotlib inline

In [2]:
cflow = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/cflow/facts',
                          allow_redirects=True).json()['data'])

In [3]:
cflow.head(10)

,amount,amount_type.code,amount_type.label,demarcation.code,demarcation.label,financial_period.period,financial_year_end.year,item.code,item.composition,item.label,item.position_in_return_form,item.return_form_structure,period_length.length
0,0,ACT,Actual,BUF,Buffalo City,1,2012,0100,None,Opening Cash Balance,NaN,None,month
1,0,ADJB,Adjusted Budget,BUF,Buffalo City,1,2012,0100,None,Opening Cash Balance,NaN,None,month
2,0,ORGB,Original Budget,BUF,Buffalo City,1,2012,0100,None,Opening Cash Balance,NaN,None,month
3,0,ACT,Actual,BUF,Buffalo City,2,2012,0100,None,Opening Cash Balance,NaN,None,month
4,0,ADJB,Adjusted Budget,BUF,Buffalo City,2,2012,0100,None,Opening Cash Balance,NaN,None,month
5,0,ORGB,Original Budget,BUF,Buffalo City,2,2012,0100,None,Opening Cash Balance,NaN,None,month
6,0,ACT,Actual,BUF,Buffalo City,3,2012,0100,None,Opening Cash Balance,NaN,None,month
7,0,ADJB,Adjusted Budget,BUF,Buffalo City,3,2012,0100,None,Opening Cash Balance,NaN,None,month
8,0,ORGB,Original Budget,BUF,Buffalo City,3,2012,0100,None,Opening Cash Balance,NaN,None,month
9,0,ACT,Actual,BUF,Buffalo City,4,2012,0100,None,Opening Cash Balance,NaN,None,month


In [4]:
cflow.groupby('item.label').count()

,amount,amount_type.code,amount_type.label,demarcation.code,demarcation.label,financial_period.period,financial_year_end.year,item.code,item.composition,item.position_in_return_form,item.return_form_structure,period_length.length
item.label,,,,,,,,,,,,
- Capital payments,36,36,36,36,36,36,36,36,0,0,0,36
- Cash and creditor payments,36,36,36,36,36,36,36,36,0,0,0,36
- Consumer deposits,36,36,36,36,36,36,36,36,0,0,0,36
- Consumer deposits repaid,36,36,36,36,36,36,36,36,0,0,0,36
- External loans received,36,36,36,36,36,36,36,36,0,0,0,36
- External loans repaid,36,36,36,36,36,36,36,36,0,0,0,36
- Grants and subsidies,36,36,36,36,36,36,36,36,0,0,0,36
- Insurance claims,36,36,36,36,36,36,36,36,0,0,0,36
- Investments made,36,36,36,36,36,36,36,36,0,0,0,36


In [5]:
bsheet = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/bsheet/facts',
                          allow_redirects=True).json()['data'])

In [6]:
bsheet.head(2)

,amount,amount_type.code,amount_type.label,demarcation.code,demarcation.label,financial_period.period,financial_year_end.year,item.code,item.composition,item.label,item.position_in_return_form,item.return_form_structure,period_length.length
0,NaN,AUDA,Audited Actual,BUF,Buffalo City,2009,2009,0100,None,COMMUNITY WEALTH / EQUITY,1,heading,year
1,NaN,AUDA,Audited Actual,BUF,Buffalo City,2010,2010,0100,None,COMMUNITY WEALTH / EQUITY,1,heading,year


In [7]:
bsheet.groupby(bsheet['item.label']).count()

,amount,amount_type.code,amount_type.label,demarcation.code,demarcation.label,financial_period.period,financial_year_end.year,item.code,item.composition,item.position_in_return_form,item.return_form_structure,period_length.length
item.label,,,,,,,,,,,,
ASSETS,90,230,230,230,230,230,230,230,0,230,230,230
Accumulated Surplus/(Deficit),345,346,346,346,346,346,346,346,0,346,346,346
Agricultural,90,230,230,230,230,230,230,230,0,230,230,230
Bank Overdraft,92,230,230,230,230,230,230,230,0,230,230,230
Biological,90,230,230,230,230,230,230,230,0,230,230,230
Borrowing,450,460,460,460,460,460,460,460,0,460,460,460
COMMUNITY WEALTH / EQUITY,135,346,346,346,346,346,346,346,0,346,346,346
Call Investment Deposits,213,230,230,230,230,230,230,230,0,230,230,230
Cash,214,230,230,230,230,230,230,230,0,230,230,230


In [8]:
pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/audit_opinions/facts',
                          allow_redirects=True).json()['data'])

,demarcation.code,demarcation.label,financial_year_end.year,opinion.code,opinion.label,opinion.report_url
0,BUF,Buffalo City,2011,adverse,Adverse opinion,http://mfma.treasury.gov.za/Documents/07.%20Au...
1,BUF,Buffalo City,2012,qualified,Qualified,http://mfma.treasury.gov.za/Documents/07.%20Au...
2,BUF,Buffalo City,2013,qualified,Qualified,http://mfma.treasury.gov.za/Documents/07.%20Au...
3,BUF,Buffalo City,2014,qualified,Qualified,http://mfma.treasury.gov.za/Documents/07.%20Au...
4,BUF,Buffalo City,2015,qualified,Qualified,http://mfma.treasury.gov.za/Documents/07.%20Au...
5,BUF,Buffalo City,2016,qualified,Qualified,http://mfma.treasury.gov.za/Documents/07.%20Au...
6,CPT,City of Cape Town,2011,unqualified_emphasis_of_matter,Unqualified - Emphasis of Matter items,http://mfma.treasury.gov.za/Documents/07.%20Au...
7,CPT,City of Cape Town,2012,unqualified_emphasis_of_matter,Unqualified - Emphasis of Matter items,http://mfma.treasury.gov.za/Documents/07.%20Au...
8,CPT,City of Cape Town,2013,unqualified,Unqualified - No findings,http://mfma.treasury.gov.za/Documents/07.%20Au...
9,CPT,City of Cape Town,2014,unqualified,Unqualified - No findings,http://mfma.treasury.gov.za/Documents/07.%20Au...


In [9]:
munis = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/municipalities/facts',
                          allow_redirects=True).json()['data'])

In [10]:
munis

,area,municipality.category,municipality.demarcation_code,municipality.fax_number,municipality.long_name,municipality.miif_category,municipality.name,municipality.parent_code,municipality.phone_number,municipality.postal_address_1,municipality.postal_address_2,municipality.postal_address_3,municipality.province_code,municipality.province_name,municipality.street_address_1,municipality.street_address_2,municipality.street_address_3,municipality.street_address_4,municipality.url
0,781026,A,BUF,043 743 8568,"Buffalo City, Eastern Cape",A,Buffalo City,EC,043 705 2000,P O BOX 134,EAST LONDON,5200,EC,Eastern Cape,Trust Bank Centre,C/O Oxford & North Street,East London,5200,http://www.buffalocity.gov.za
1,3740031,A,CPT,None,"City of Cape Town, Western Cape",A,City of Cape Town,WC,021 400 1111,PRIVATE BAG X9181,CAPE TOWN,8000,WC,Western Cape,Civic Centre,12 Hertzog Boulevard,Cape Town,8001,http://www.capetown.gov.za
2,391766,C,DC1,086 692 6113,"West Coast, Western Cape",C1,West Coast,WC,022 433 8400,P O BOX 242,MOORREESBURG,7310,WC,Western Cape,58 Long Street,Moorreesburg,7310,None,http://www.westcoastdm.co.za
3,450585,C,DC10,041 508 7022,"Cacadu, Eastern Cape",C1,Cacadu,EC,041 508 7111,P O BOX 318,PORT ELIZABETH,6000,EC,Eastern Cape,Standard Bank Building,32 Govan Mbeki Avenue,Port Elizabeth,6000,http://www.cacadu.co.za
4,855793,C,DC12,043 742 0337,"Amathole, Eastern Cape",C2,Amathole,EC,043 701 4000,P O BOX 320,EAST LONDON,5200,EC,Eastern Cape,Waverly Office Park,"3-33 Philli Frame RD, Chiselhurst",East London,5201,http://www.amathole.gov.za
5,806478,C,DC13,045 839 2355,"Chris Hani, Eastern Cape",C2,Chris Hani,EC,045 808 4600,PRIVATE BAG X7121,QUEENSTOWN,5320,EC,Eastern Cape,15 Bells Road,Queenstown,5320,None,http://www.chrishanidm.gov.za
6,348667,C,DC14,045 979 3028,"Joe Gqabi, Eastern Cape",C2,Joe Gqabi,EC,045 979 3000,P O BOX 102,BARKLY EAST,9786,EC,Eastern Cape,Joe Gqabi District Municipality,C/ O Graham & Cole Street,Barkly East,9786,http://www.jgdm.gov.za
7,1366044,C,DC15,047 532 4166,"O.R.Tambo, Eastern Cape",C2,O.R.Tambo,EC,047 501 6400,PRIVAATE BAG X6043,MTHATHA,5099,EC,Eastern Cape,O R Tambo House,Nelson Mandela Drive,Mthatha,5099,http://www.ortambodm.org.za
8,121945,C,DC16,051 713 0461,"Xhariep, Free State",C1,Xhariep,FS,051 713 9300,PRIVATE BAG X136,TROMPSBURG,9913,FS,Free State,Xhariep District Municipality,20 Louw Street,Trompsburg,9913,http://www.xhariep.fs.gov.za
9,624746,C,DC18,057 357 4103,"Lejweleputswa, Free State",C1,Lejweleputswa,FS,057 353 3094,P O BOX 2163,WELKOM,9460,FS,Free State,Lejwelputswa District Municipality,Cnr Tempest & Jan Hofmeyer Road,Welkom,9460,http://www.lejwe.co.za


In [11]:
cubes = requests.get('https://municipaldata.treasury.gov.za/api/cubes/', allow_redirects=True).json()

In [12]:
data = pd.DataFrame(cubes['data'])

In [13]:
data

,description,label,name,uri
0,Analysis of creditors aggregated by age of deb...,Aged Creditor Analysis,aged_creditor,https://municipaldata.treasury.gov.za/api/cube...
1,Details of grants received from national gover...,Conditional Grants,conditional_grants,https://municipaldata.treasury.gov.za/api/cube...
2,Statement of Financial Performance. How a muni...,Income and Expenditure,incexp,https://municipaldata.treasury.gov.za/api/cube...
3,Analysis of debtors by age of debt. Does not i...,Aged Debtor Analysis,aged_debtor,https://municipaldata.treasury.gov.za/api/cube...
4,"Purchase, repair and renewal of capital assets.",Capital Acquisition,capital,https://municipaldata.treasury.gov.za/api/cube...
5,Specific expenditure amounts from audited fina...,"Unauthorised, Irregular, Fruitless and Wastefu...",uifwexp,https://municipaldata.treasury.gov.za/api/cube...
6,The movement of cash into and out of the munic...,Cash Flow,cflow,https://municipaldata.treasury.gov.za/api/cube...
7,General information about the municipality inc...,Municipalities,municipalities,https://municipaldata.treasury.gov.za/api/cube...
8,Outcomes of financial statement audits from th...,Audit Opinions,audit_opinions,https://municipaldata.treasury.gov.za/api/cube...
9,Names and contact information for high level m...,Municipal Officials,officials,https://municipaldata.treasury.gov.za/api/cube...


In [14]:
aged = requests.get(data.uri[0])

In [15]:
def url_change(x, url = 'https://municipaldata.treasury.gov.za/api/cubes/'):
    return ''.join([url,x.split(url)[1].replace('/','')])

In [16]:
data.uri= data.uri.apply(lambda x: url_change(x))

In [17]:
test = requests.get(data.uri[7], allow_redirects=True).json()

In [18]:
data

,description,label,name,uri
0,Analysis of creditors aggregated by age of deb...,Aged Creditor Analysis,aged_creditor,https://municipaldata.treasury.gov.za/api/cube...
1,Details of grants received from national gover...,Conditional Grants,conditional_grants,https://municipaldata.treasury.gov.za/api/cube...
2,Statement of Financial Performance. How a muni...,Income and Expenditure,incexp,https://municipaldata.treasury.gov.za/api/cube...
3,Analysis of debtors by age of debt. Does not i...,Aged Debtor Analysis,aged_debtor,https://municipaldata.treasury.gov.za/api/cube...
4,"Purchase, repair and renewal of capital assets.",Capital Acquisition,capital,https://municipaldata.treasury.gov.za/api/cube...
5,Specific expenditure amounts from audited fina...,"Unauthorised, Irregular, Fruitless and Wastefu...",uifwexp,https://municipaldata.treasury.gov.za/api/cube...
6,The movement of cash into and out of the munic...,Cash Flow,cflow,https://municipaldata.treasury.gov.za/api/cube...
7,General information about the municipality inc...,Municipalities,municipalities,https://municipaldata.treasury.gov.za/api/cube...
8,Outcomes of financial statement audits from th...,Audit Opinions,audit_opinions,https://municipaldata.treasury.gov.za/api/cube...
9,Names and contact information for high level m...,Municipal Officials,officials,https://municipaldata.treasury.gov.za/api/cube...


In [19]:
requests.get(test['endpoints'][2], allow_redirects=True).json()

{'cell': None,
 'data': [{'area': 781026,
   'municipality.category': 'A',
   'municipality.demarcation_code': 'BUF',
   'municipality.fax_number': '043 743 8568',
   'municipality.long_name': 'Buffalo City, Eastern Cape',
   'municipality.miif_category': 'A',
   'municipality.name': 'Buffalo City',
   'municipality.parent_code': 'EC',
   'municipality.phone_number': '043 705 2000',
   'municipality.postal_address_1': 'P O BOX 134',
   'municipality.postal_address_2': 'EAST LONDON',
   'municipality.postal_address_3': '5200',
   'municipality.province_code': 'EC',
   'municipality.province_name': 'Eastern Cape',
   'municipality.street_address_1': 'Trust Bank Centre',
   'municipality.street_address_2': 'C/O Oxford & North Street',
   'municipality.street_address_3': 'East London',
   'municipality.street_address_4': '5200',
   'municipality.url': 'http://www.buffalocity.gov.za'},
  {'area': 3740031,
   'municipality.category': 'A',
   'municipality.demarcation_code': 'CPT',
   'munici

In [20]:
args = urlencode({'municipality.code':"BUF"})

In [21]:
target_cashflow = 'https://municipaldata.treasury.gov.za/api/cubes/cflow/facts'

In [22]:
target_munis = 'https://municipaldata.treasury.gov.za/api/cubes//facts'

In [23]:
target_officials = 'https://municipaldata.treasury.gov.za/api/cubes/officials/facts'

In [24]:
munis = requests.get(target_munis,allow_redirects=True).json()['data']

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [ ]:
officials = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/officials/facts',
                                      allow_redirects=True).json()['data'])
#cashflows = pd.DataFrame(requests.get(target_cashflow,allow_redirects=True).json()['data'])

In [ ]:
officials

In [ ]:
municipalities = []

for muni in munis:
    temp = {}
    temp['municipality.demarcation_code'] = muni['municipality.demarcation_code']
    temp['name'] = muni['municipality.name']
    temp['province'] = muni['municipality.province_name']
    temp['category'] = muni['municipality.miif_category']
    temp['area'] = muni['area']
    municipalities.append(temp)
municipalities = pd.DataFrame(municipalities)

In [ ]:
officials = officials['data']

In [ ]:
officials = pd.DataFrame(officials)

In [ ]:
officials

In [ ]:
municipalities.columns

In [ ]:
grants = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/conditional_grants/facts',
                                      allow_redirects=True).json()['data'])

In [ ]:
grants

In [ ]:
pd.merge(municipalities, officials, how='inner', on='municipality.demarcation_code').groupby('contact_details.title').count()

In [ ]:
capital = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/capital/facts',
                          allow_redirects=True).json()['data'])

In [ ]:
capital.head(5)